## 143: Autenticación con Múltiples Usuarios y Roles desde Base de Datos

Hoy aprenderás a almacenar y autenticar múltiples usuarios con roles en una base de datos (usando SQLAlchemy), en lugar de usar un usuario fijo. Esto te permitirá:

👥 Gestionar múltiples cuentas

🔒 Asignar roles como admin, usuario, report_viewer

🧩 Autenticación real con validación de contraseña hasheada

🧩 1. Instalar dependencias necesarias
bash
Copiar
Editar
pip install sqlalchemy sqlite3 passlib
🧩 2. Crear el modelo User con SQLAlchemy
python
Copiar
Editar
# models.py
from sqlalchemy import Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from passlib.context import CryptContext

Base = declarative_base()
pwd_context = CryptContext(schemes=["bcrypt"], deprecated="auto")

class User(Base):
    __tablename__ = "usuarios"
    id = Column(Integer, primary_key=True, index=True)
    username = Column(String, unique=True)
    email = Column(String, unique=True)
    hashed_password = Column(String)
    rol = Column(String)

    def verificar_password(self, password: str):
        return pwd_context.verify(password, self.hashed_password)

    @staticmethod
    def hashear_password(password: str):
        return pwd_context.hash(password)
🧩 3. Crear la base de datos SQLite y añadir usuarios
python
Copiar
Editar
# db_setup.py
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from models import Base, User

engine = create_engine("sqlite:///usuarios.db")
Base.metadata.create_all(engine)

Session = sessionmaker(bind=engine)
db = Session()

# Crear un usuario admin
nuevo_usuario = User(
    username="admin",
    email="admin@correo.com",
    hashed_password=User.hashear_password("1234"),
    rol="admin"
)
db.add(nuevo_usuario)
db.commit()
🧩 4. Actualizar login para autenticar desde la BD
python
Copiar
Editar
# main.py
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine
from models import User
from auth import crear_token, verificar_token
from fastapi.security import OAuth2PasswordRequestForm
from fastapi import FastAPI, Depends, HTTPException

engine = create_engine("sqlite:///usuarios.db")
Session = sessionmaker(bind=engine)
db = Session()

app = FastAPI()

@app.post("/login")
def login(form_data: OAuth2PasswordRequestForm = Depends()):
    user = db.query(User).filter_by(username=form_data.username).first()
    if not user or not user.verificar_password(form_data.password):
        raise HTTPException(status_code=401, detail="Credenciales inválidas")

    token = crear_token({"sub": user.username, "rol": user.rol})
    return {"access_token": token, "token_type": "bearer"}
🧩 5. Proteger rutas por rol
python
Copiar
Editar
from auth import verificar_token
from fastapi import Depends

@app.post("/enviar-reporte/")
def enviar_reporte(data: ReporteRequest, usuario: dict = Depends(verificar_token)):
    if usuario["rol"] != "admin":
        raise HTTPException(status_code=403, detail="Solo administradores pueden enviar reportes.")
    ...